# Lab: Create Qdrant (a vector database)
## Vector
- Sparse Vector: eg. one-hot encoding 編碼 categorical data
  - 詞彙量增加 => 維度增加
- Dense Vecotr: 於 NLP 的應用
  - 這些向量能夠捕獲單詞的語義，並且將語義相近的單詞投影到嵌入空間中的相近位置
  - 可以捕獲單詞之間的相關性
- Co-occurrence: 共現或共現是文本語料庫中兩個相鄰術語有序出現的高機率頻率
  - 找到語意相似
- Cosine metrics: 判斷相似性，2 vectors 夾角 cos 值越接近 1 越相似 (0: 90度 獨立)
  - Cosine distance: 0 表示完全相同, 2 表示完全不同
    - Cosine distance(A,B) = 1 - Cosine Similarity(A,B)

## Qdrant 重要名詞
- Collections（集合）：集合是帶有名稱的一堆 points（附帶 payload 的向量），可以在其中進行搜索。可以想像成傳統資料庫的 table
- Payload：用來與向量一起儲存額外的資訊，就是 metadata 的概念
- Point：Point 是 Qdrant 儲存資料的核心實體。一個點是由向量和 Payload 所組成的

## Qdrant vector database
- `docker-compose.yml`:
```
services:
  qdrant:
    image: qdrant/qdrant:latest
    restart: always
    ports:
      - '6333:6333'
    volumes: # local:docker
      - ./qdrant_storage:/qdrant/storage
```
- 使用指令 `sudo docker compose up -d` -d: run background (read `docker-compose.yml` on the folder automatically)
- 進到 http://localhost:6333/ 檢查 

- 安裝 Qdrant 的 Python SDK
- 使用指令 `poetry add qdrant-client`

# Lab: qdrant_tutorial
- `poetry add qdrant-client`
- `http://<ip>:6333/dashboard` 查看資料庫
- CRUD

In [ ]:
# Create a Qdrant collection
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Localhost example
# client = QdrantClient("localhost", port=6333)

# Cloud example
client = QdrantClient(
    url="https://eeaa6571-6f65-4174-a647-9091668bb8c0.us-west-1-0.aws.cloud.qdrant.io:6333", 
    api_key="",
)

client.create_collection(
    collection_name="test",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.COSINE), # size: vector 維度
)

In [ ]:
# Recreate collection (table)
client.recreate_collection(
    collection_name="test",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.COSINE),
)

In [ ]:
# delete collection
client.delete_collection(collection_name="test")

# 注意要 1.7 之後的 qdrant 才支援 collection_exists method
if not client.collection_exists(collection_name="test"):
    client.create_collection(
        collection_name="test",
        vectors_config=models.VectorParams(size=3, distance=models.Distance.COSINE),
    )
    print("Not exisit, collection created")

In [ ]:
# upsert (insert) 插入一個點位 (point)
client.upsert(
    collection_name="test",
    points=[
        models.PointStruct(
            id="5c56c793-69f3-4fbf-87e6-c4bf54c28c26", # uuid
            payload={
                "color": "red", # metadata
            },
            vector=[0.9, 0.1, 0.1], # word vector
        ),
    ],
)

In [ ]:
# update 更新向量
client.update_vectors(
    collection_name="test",
    points=[
        models.PointVectors(
            id="5c56c793-69f3-4fbf-87e6-c4bf54c28c26",
            vector=[0.9, 0.9, 0.9],
        ),
    ]
)

In [ ]:
# delete point 刪除特定 point
client.delete(
    collection_name="test",
    points_selector=models.PointIdsList(
        points=["5c56c793-69f3-4fbf-87e6-c4bf54c28c26"],
    ),
)

In [ ]:
# set_payload 更新 payload (metadata)
client.set_payload(
    collection_name="test",
    payload={
        "property1": "string",
        "property2": "string",
    },
    points=["5c56c793-69f3-4fbf-87e6-c4bf54c28c26"],
)

In [ ]:
# overwrite_payload 複寫 payload
client.overwrite_payload(
    collection_name="test",
    payload={
        "property1": "string",
        "property2": "string",
    },
    points=["5c56c793-69f3-4fbf-87e6-c4bf54c28c26"],
)

In [ ]:
# delete_payload 刪除 payload 指定 key
client.delete_payload(
    collection_name="test",
    keys=["property2"],
    points=["5c56c793-69f3-4fbf-87e6-c4bf54c28c26"],
)

# LangChain 整合 Qdrant

In [ ]:
# Connect to Ollama Gemma LLM using LangChain
from langchain_ollama import OllamaLLM

# Initialize the Ollama LLM with your deployed Gemma model
ollama_llm = OllamaLLM(
    base_url="http://192.168.72.20:11434",  # Your Ollama server URL
    model="gemma3n:e2b",  # Model name (adjust if your model has a different name)
    temperature=0.7,  # Adjust temperature for creativity (0.0 to 1.0)
)

# Test the connection with a simple query
test_query = "Hello, can you introduce yourself?"
response = ollama_llm.invoke(test_query)
print(f"Query: {test_query}")
print(f"Response: {response}")

In [ ]:
# 使用 Langchain 的 Qdrant 套件處裡文字向量
from langchain_community.vectorstores import Qdrant # <--
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(
    base_url="http://192.168.72.20:11434",
    model="nomic-embed-text:v1.5",
)


data_objs = [
    {
        "id": 1,
        "lyric": "我會披星戴月的想你，我會奮不顧身的前進，遠方煙火越來越唏噓，凝視前方身後的距離"
    },
    {
        "id": 2,
        "lyric": "而我，在這座城市遺失了你，順便遺失了自己，以為荒唐到底會有捷徑。而我，在這座城市失去了你，輸給慾望高漲的自己，不是你，過分的感情"
    }
]

lyric_list = [data_obj["lyric"] for data_obj in data_objs]

qdrant = Qdrant.from_texts(
    lyric_list,  # list contained lyric
    embeddings_model,  # model
    url="https://eeaa6571-6f65-4174-a647-9091668bb8c0.us-west-1-0.aws.cloud.qdrant.io:6333",
    collection_name="test",
    force_recreate=True,
    api_key="",
)

output = qdrant.similarity_search(query="工程師寫城市", k=1, )
# output = qdrant.similarity_search(query="我離你愈來愈遠", k=1, )
print(output)

In [ ]:
# Advanced usage: Combine Ollama LLM with Qdrant for RAG (Retrieval Augmented Generation)
from langchain_ollama import OllamaEmbeddings

# Initialize Ollama embeddings (you can use a different model for embeddings)
ollama_embeddings = OllamaEmbeddings(
    base_url="http://192.168.72.20:11434",
    model="gemma3n",  # Or use a different model optimized for embeddings if available
)

# Example: Using both Ollama LLM and Qdrant for document search and generation
def search_and_generate(query, qdrant_store, llm):
    """
    Search for relevant documents in Qdrant and generate an answer using Ollama LLM
    """
    # Search for similar documents
    similar_docs = qdrant_store.similarity_search(query, k=3)
    
    # Combine the found documents as context
    context = "\n".join([doc.page_content for doc in similar_docs])
    
    # Create a prompt with context
    prompt = f"""
    Based on the following context, please answer the question:
    
    Context:
    {context}
    
    Question: {query}
    
    Answer:
    """
    
    # Generate response using Ollama LLM
    response = llm.invoke(prompt)
    return response, similar_docs

# Example usage (uncomment when you have a Qdrant store ready)
query = "Tell me about the lyrics"
qdrant = Qdrant.from_texts(
    lyric_list,  # list contained lyric
    embeddings_model,  # model
    url="https://eeaa6571-6f65-4174-a647-9091668bb8c0.us-west-1-0.aws.cloud.qdrant.io:6333",
    collection_name="test",
    force_recreate=True,
    api_key="",
)

ollama_llm = OllamaLLM(
    base_url="http://192.168.72.20:11434",  # Your Ollama server URL
    model="gemma3n:e2b",  # Model name (adjust if your model has a different name)
    temperature=0.7,  # Adjust temperature for creativity (0.0 to 1.0)
)

answer, docs = search_and_generate(query, qdrant, ollama_llm)
print(f"Answer: {answer}")
print(f"Source documents: {len(docs)} documents found")